In [3]:
import argparse
import math
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM
from advertorch.attacks import LinfSPSAAttack
from src.trainers import Trainer, FGSMTrainer
from robustbench.model_zoo.models import Carmon2019UnlabeledNet
from src.utils import adversarial_accuracy, fgsm_
from src.gradient_masking_tests import gradient_norm, fgsm_pgd_cos_dif, multi_scale_fgsm, gradient_information, linearization_error
import eagerpy as ep
from src.Nets import CIFAR_Wide_Res_Net, CIFAR_Res_Net, CIFAR_Net, CUREResNet18
%load_ext autoreload
%autoreload 2
%aimport src.Nets, src.trainers, src.gradient_masking_tests

In [4]:
# setup
device = torch.device("cuda")
batch_size = 128
# remove the normalize
transform = transform = transforms.Compose(
            [transforms.ToTensor()]
)
        
normalized_min = (0 - 0.5) / 0.5
normalized_max = (1 - 0.5) / 0.5
train_dataset = datasets.CIFAR10(root='../data', train=True,
                                download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                  shuffle=True, num_workers=2)
test_dataset = datasets.CIFAR10(root='../data', train=False,
                               download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)
classes = classes = ('plane', 'car', 'bird', 'cat',
   'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load CIFAR-10 models

In [10]:
# # Regular CIFAR-10  ResNet Model
model = CIFAR_Res_Net().eval().to(device)
model.load_state_dict(torch.load("../models/cifar_res_net.model"))
#CIFAR-10  ResNet Model trained with pgd 03
pgd_model = CIFAR_Res_Net().to(device).eval()
pgd_model.load_state_dict(torch.load("../models/cifar_res_net_pgd_20e_03.model"))
#CIFAR-10  ResNet Model trained with pgd 06
pgd_model_6 = CIFAR_Res_Net().to(device).eval()
pgd_model_6.load_state_dict(torch.load("../models/cifar_res_net_pgd_20e_06.model"))
#CIFAR-10  ResNet Model trained with large FGSM steps
fgsm_model = CIFAR_Res_Net().to(device).eval()
fgsm_model.load_state_dict(torch.load("../models/cifar_res_net_fgsm06.model"))
# # # CIFAR-10  ResNet Model trained with small FGSM steps
fgsm_model_small = CIFAR_Res_Net().to(device).eval()
fgsm_model_small.load_state_dict(torch.load("../models/cifar_res_net_20e_fgsm03.model"))
# # # CIFAR-10  ResNet Model trained with large Step-ll steps
step_ll_model = CIFAR_Res_Net().to(device).eval()
step_ll_model.load_state_dict(torch.load("../models/cifar_res_net_20e_stepll.model"))
# # # CIFAR-10  ResNet Model trained with small Step-ll steps
step_ll_model_small = CIFAR_Res_Net().to(device).eval()
step_ll_model_small.load_state_dict(torch.load("../models/cifar_res_net_step_ll_20e_03.model"))
# # CIFAR-10  ResNet Model trained through Jacobian regularization ld0.1
jac_norm_model = CIFAR_Res_Net().to(device).eval()
jac_norm_model.load_state_dict(torch.load("../models/cifar_res_net_jac_reg_20e_ld_01.model"))
## Pretrained CIFAR-10 RESNET trained using CURE
cure = CUREResNet18().to(device).eval()
cure[1].load_state_dict(torch.load("../models/RN18_CURE.pth")['net'])

<All keys matched successfully>

### Check gradient norms
Gradients seem to blow up after FGSM training

In [11]:
subset_size = 5000
train_subset = torch.utils.data.Subset(train_dataset, np.random.randint(0, len(train_dataset), size=subset_size).tolist())
train_subset_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)

test_subset = torch.utils.data.Subset(test_dataset, np.random.randint(0, len(test_dataset), size=subset_size).tolist())
test_subset_loader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)

In [12]:
def show_grad_histogram(model, data_loader, device='cpu'):
    norm = gradient_norm(model, data_loader, device=device).detach().cpu().numpy()
    plt.hist(norm, bins=100)
    plt.show()
    print("Max: {}, Min: {}, Mean: {}, Median: {}".format(norm.max(), norm.min(), norm.mean(), np.median(norm)))

def compare_models_grad(models, labels, data_loader, device='cpu', height=2):
    norms = [gradient_norm(model, data_loader, device=device).detach().cpu().numpy() for model in models]
    width = math.ceil(len(models)/height)
    fig, ax = plt.subplots(height, width, figsize=(15, 15))
    for index, norm in enumerate(norms):
        axis = ax[index//width, index%width]
        axis.hist(norm, bins=100)
        axis.set_title(labels[index])
        axis.text(0.5, -0.13, "Max: {:.6f}, Min: {:.6f}, \nMean: {:.6f}, Median: {:.6f}".format(norm.max().item(), norm.min().item(), norm.mean().item(), np.median(norm).item()),
                                            size=12, ha="center", transform=axis.transAxes)
    plt.show()

### Using train dataset

In [ ]:
compare_models_grad([model, fgsm_model, fgsm_model_small, pgd_model, pgd_model_6, jac_norm_model, step_ll_model, step_ll_model_small, cure], ['Normal', 'FGSM', 'FGSM Small', 'PGD', 'PGD Small', 'Jacobian Reg', 'Step-ll', 'Step-ll Small', 'CURE'], train_subset_loader, device=device, height=3)

### Using test dataset

In [ ]:
compare_models_grad([model, fgsm_model, fgsm_model_small, pgd_model, pgd_model_6, jac_norm_model, step_ll_model, step_ll_model_small, cure], ['Normal', 'FGSM', 'FGSM Small', 'PGD', 'PGD Small', 'Jacobian Reg', 'Step-ll', 'Step-ll Small', 'CURE'], test_subset_loader, device=device, height=3)

## Cosine difference between FGSM and PGD

### Using train dataset

#### Normal model

In [6]:
fgsm_pgd_cos_dif(model, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.3335760533809662, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.2772107720375061, Mean l2 Distance: 1.7560908794403076
FGSM success: 0.9501999616622925, PGD Success: 1.0, Rescaled FGSM success: 0.939799964427948



Epsilon = 0.06:
Mean Cosine Difference: 0.204548642039299, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.1350550800561905, Mean l2 Distance: 3.771761178970337
FGSM success: 0.9829999804496765, PGD Success: 1.0, Rescaled FGSM success: 0.9783999919891357


#### FGSM Model

In [8]:
fgsm_pgd_cos_dif(fgsm_model, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.09495557099580765, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.07238291203975677, Mean l2 Distance: 1.93947434425354
FGSM success: 0.13699999451637268, PGD Success: 1.0, Rescaled FGSM success: 0.46959999203681946



Epsilon = 0.06:
Mean Cosine Difference: 0.029067358002066612, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.030301837250590324, Mean l2 Distance: 3.941718578338623
FGSM success: 0.0543999969959259, PGD Success: 1.0, Rescaled FGSM success: 0.04099999740719795


#### FGSM Model small steps

In [7]:
fgsm_pgd_cos_dif(fgsm_model_small, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.6148859858512878, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5915178656578064, Mean l2 Distance: 1.3246660232543945
FGSM success: 0.16599999368190765, PGD Success: 0.2507999837398529, Rescaled FGSM success: 0.1607999950647354



Epsilon = 0.06:
Mean Cosine Difference: 0.4654251039028168, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.4688422977924347, Mean l2 Distance: 3.2853798866271973
FGSM success: 0.5525999665260315, PGD Success: 0.8509999513626099, Rescaled FGSM success: 0.515999972820282


#### PGD Model

In [15]:
fgsm_pgd_cos_dif(pgd_model_6, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.735053539276123, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.7200735807418823, Mean l2 Distance: 1.2345870733261108
FGSM success: 0.5026000142097473, PGD Success: 0.5113999843597412, Rescaled FGSM success: 0.5005999803543091



Epsilon = 0.06:
Mean Cosine Difference: 0.6044542789459229, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5940747857093811, Mean l2 Distance: 2.7045068740844727
FGSM success: 0.616599977016449, PGD Success: 0.6759999990463257, Rescaled FGSM success: 0.6083999872207642


#### PGD Model small steps

In [16]:
fgsm_pgd_cos_dif(pgd_model, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.629551887512207, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5887507200241089, Mean l2 Distance: 1.4309372901916504
FGSM success: 0.1785999983549118, PGD Success: 0.2629999816417694, Rescaled FGSM success: 0.17479999363422394



Epsilon = 0.06:
Mean Cosine Difference: 0.4690273106098175, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.47609037160873413, Mean l2 Distance: 3.121513843536377
FGSM success: 0.5090000033378601, PGD Success: 0.8068000078201294, Rescaled FGSM success: 0.47839999198913574


#### Gradient Regularization Model

In [17]:
fgsm_pgd_cos_dif(jac_norm_model, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.6795960664749146, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.7039350271224976, Mean l2 Distance: 1.1962990760803223
FGSM success: 0.7325999736785889, PGD Success: 0.7907999753952026, Rescaled FGSM success: 0.7245999574661255



Epsilon = 0.06:
Mean Cosine Difference: 0.48964768648147583, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5596106648445129, Mean l2 Distance: 3.055312156677246
FGSM success: 0.9345999956130981, PGD Success: 0.9839999675750732, Rescaled FGSM success: 0.9273999929428101


#### Step-ll Model

In [8]:
fgsm_pgd_cos_dif(step_ll_model, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.24079076945781708, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.22432973980903625, Mean l2 Distance: 1.8763231039047241
FGSM success: 0.006199999712407589, PGD Success: 0.8769999742507935, Rescaled FGSM success: 0.020600000396370888



Epsilon = 0.06:
Mean Cosine Difference: 0.14389684796333313, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.12367715686559677, Mean l2 Distance: 3.8352019786834717
FGSM success: 0.006399999838322401, PGD Success: 0.9923999905586243, Rescaled FGSM success: 0.0037999998312443495


TypeError: cannot unpack non-iterable NoneType object

In [21]:
fgsm_pgd_cos_dif(step_ll_model_small, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.6243112087249756, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5855385065078735, Mean l2 Distance: 1.5512077808380127
FGSM success: 0.280599981546402, PGD Success: 0.5163999795913696, Rescaled FGSM success: 0.25920000672340393



Epsilon = 0.06:
Mean Cosine Difference: 0.4667837917804718, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.3519720137119293, Mean l2 Distance: 3.220111131668091
FGSM success: 0.7691999673843384, PGD Success: 0.9729999899864197, Rescaled FGSM success: 0.7505999803543091


In [23]:
fgsm_pgd_cos_dif(cure, train_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.7266350388526917, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.7165587544441223, Mean l2 Distance: 1.1422247886657715
FGSM success: 0.4477999806404114, PGD Success: 0.5045999884605408, Rescaled FGSM success: 0.4399999976158142



Epsilon = 0.06:
Mean Cosine Difference: 0.5679991841316223, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5807271003723145, Mean l2 Distance: 2.9217920303344727
FGSM success: 0.8082000017166138, PGD Success: 0.914199948310852, Rescaled FGSM success: 0.7945999503135681


### Using test dataset

#### Normal Model

In [17]:
fgsm_pgd_cos_dif(model, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.35395342111587524, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.2613288462162018, Mean l2 Distance: 1.7340483665466309
FGSM success: 0.9508000016212463, PGD Success: 1.0, Rescaled FGSM success: 0.9411999583244324



Epsilon = 0.06:
Mean Cosine Difference: 0.2228918820619583, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.1459217071533203, Mean l2 Distance: 3.7200064659118652
FGSM success: 0.9819999933242798, PGD Success: 1.0, Rescaled FGSM success: 0.9789999723434448


#### FGSM Model

In [18]:
fgsm_pgd_cos_dif(fgsm_model, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.08936622738838196, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.07271866500377655, Mean l2 Distance: 1.9675538539886475
FGSM success: 0.2590000033378601, PGD Success: 1.0, Rescaled FGSM success: 0.5521999597549438



Epsilon = 0.06:
Mean Cosine Difference: 0.03259320184588432, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.029694553464651108, Mean l2 Distance: 3.8834316730499268
FGSM success: 0.14219999313354492, PGD Success: 1.0, Rescaled FGSM success: 0.14019998908042908


#### PGD Model

In [18]:
fgsm_pgd_cos_dif(pgd_model_6, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.7403298020362854, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.7208808064460754, Mean l2 Distance: 1.171081781387329
FGSM success: 0.589199960231781, PGD Success: 0.6001999974250793, Rescaled FGSM success: 0.5884000062942505



Epsilon = 0.06:
Mean Cosine Difference: 0.6173502206802368, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5966373085975647, Mean l2 Distance: 2.7057061195373535
FGSM success: 0.7073999643325806, PGD Success: 0.7577999830245972, Rescaled FGSM success: 0.7017999887466431


#### PGD Model small steps

In [19]:
fgsm_pgd_cos_dif(pgd_model, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.6655188202857971, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.6270831227302551, Mean l2 Distance: 1.2524887323379517
FGSM success: 0.6273999810218811, PGD Success: 0.6797999739646912, Rescaled FGSM success: 0.6215999722480774



Epsilon = 0.06:
Mean Cosine Difference: 0.5185705423355103, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.4975077509880066, Mean l2 Distance: 3.373537063598633
FGSM success: 0.8255999684333801, PGD Success: 0.9177999496459961, Rescaled FGSM success: 0.8143999576568604


#### Gradient Regularization Model

In [20]:
fgsm_pgd_cos_dif(jac_norm_model, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.681378185749054, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.6890079975128174, Mean l2 Distance: 1.242175579071045
FGSM success: 0.7545999884605408, PGD Success: 0.8053999543190002, Rescaled FGSM success: 0.7477999925613403



Epsilon = 0.06:
Mean Cosine Difference: 0.48985564708709717, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5543279051780701, Mean l2 Distance: 3.419525623321533
FGSM success: 0.9369999766349792, PGD Success: 0.9813999533653259, Rescaled FGSM success: 0.9315999746322632


#### Step-ll Model

In [21]:
fgsm_pgd_cos_dif(step_ll_model, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.23231205344200134, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.2298101782798767, Mean l2 Distance: 1.8700395822525024
FGSM success: 0.41920000314712524, PGD Success: 0.9339999556541443, Rescaled FGSM success: 0.45339998602867126



Epsilon = 0.06:
Mean Cosine Difference: 0.1347338706254959, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.13164548575878143, Mean l2 Distance: 3.905555248260498
FGSM success: 0.45659998059272766, PGD Success: 0.9945999979972839, Rescaled FGSM success: 0.4262000024318695


In [22]:
fgsm_pgd_cos_dif(step_ll_model_small, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.6360668540000916, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5948304533958435, Mean l2 Distance: 1.4246094226837158
FGSM success: 0.6943999528884888, PGD Success: 0.7651999592781067, Rescaled FGSM success: 0.6843999624252319



Epsilon = 0.06:
Mean Cosine Difference: 0.490384966135025, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.34859323501586914, Mean l2 Distance: 3.3363327980041504
FGSM success: 0.8727999925613403, PGD Success: 0.9817999601364136, Rescaled FGSM success: 0.8643999695777893


In [24]:
fgsm_pgd_cos_dif(cure, test_dataset, epsilons=[0.03, 0.06], device=device, batch_size=batch_size, subset_size=5000, n_steps_pgd=20)


Epsilon = 0.03:
Mean Cosine Difference: 0.7333496809005737, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.731689453125, Mean l2 Distance: 1.170149326324463
FGSM success: 0.5557999610900879, PGD Success: 0.6047999858856201, Rescaled FGSM success: 0.5489999651908875



Epsilon = 0.06:
Mean Cosine Difference: 0.5726946592330933, Mean Cosine Difference when FGSM does not succeed but PGD does: 0.5840405821800232, Mean l2 Distance: 3.022047996520996
FGSM success: 0.8145999908447266, PGD Success: 0.913599967956543, Rescaled FGSM success: 0.8075999617576599


## Linearizability of Model

#### Training data

In [33]:
for cur_model, name in zip([model, step_ll_model, step_ll_model_small, fgsm_model, fgsm_model_small, pgd_model_6, pgd_model, jac_norm_model, cure], ['normal', 'step ll', 'step ll small', 'fgsm', 'fgsm small', 'pgd', 'pgd small', 'grad norm', 'cure']):
    print(name + ': ')
    linearization_error(cur_model, train_dataset, epsilons=[0.003, 0.03, 0.06, 0.1], subset=1000, batch_size=32, n_perturbations=32, device=device)

normal: 
Epsilon 0.003: 0.03307342529296875 error
Epsilon 0.03: 0.3565625548362732 error
Epsilon 0.06: 1.566001057624817 error
Epsilon 0.1: 8.340760231018066 error
step ll: 
Epsilon 0.003: 0.023074664175510406 error
Epsilon 0.03: 0.2147114872932434 error
Epsilon 0.06: 0.383059561252594 error
Epsilon 0.1: 0.7022789716720581 error
step ll small: 
Epsilon 0.003: 0.006576708983629942 error
Epsilon 0.03: 0.06567896157503128 error
Epsilon 0.06: 0.1467602550983429 error
Epsilon 0.1: 0.32411131262779236 error
fgsm: 
Epsilon 0.003: 0.14247646927833557 error
Epsilon 0.03: 1.473429799079895 error
Epsilon 0.06: 2.972611665725708 error
Epsilon 0.1: 23.294282913208008 error
fgsm small: 
Epsilon 0.003: 0.00502273952588439 error
Epsilon 0.03: 0.04990667477250099 error
Epsilon 0.06: 0.14848743379116058 error
Epsilon 0.1: 0.24841904640197754 error
pgd: 
Epsilon 0.003: 0.00624723918735981 error
Epsilon 0.03: 0.06239868700504303 error
Epsilon 0.06: 0.262955904006958 error
Epsilon 0.1: 0.3219253718852997 e

#### Testing data

In [34]:
for cur_model, name in zip([model, step_ll_model, step_ll_model_small, fgsm_model, fgsm_model_small, pgd_model_6, pgd_model, jac_norm_model, cure], ['normal', 'step ll', 'step ll small', 'fgsm', 'fgsm small', 'pgd', 'pgd small', 'grad norm', 'cure']):
    print(name + ': ')
    linearization_error(cur_model, test_dataset, epsilons=[0.003, 0.03, 0.06, 0.1], subset=1000, batch_size=32, n_perturbations=32, device=device)

normal: 
Epsilon 0.003: 0.0717787966132164 error
Epsilon 0.03: 1.0048729181289673 error
Epsilon 0.06: 5.74287748336792 error
Epsilon 0.1: 16.59425926208496 error
step ll: 
Epsilon 0.003: 0.05241377279162407 error
Epsilon 0.03: 0.6117618680000305 error
Epsilon 0.06: 1.2578635215759277 error
Epsilon 0.1: 3.018287420272827 error
step ll small: 
Epsilon 0.003: 0.0216224268078804 error
Epsilon 0.03: 0.17331956326961517 error
Epsilon 0.06: 0.752302348613739 error
Epsilon 0.1: 1.4778895378112793 error
fgsm: 
Epsilon 0.003: 0.18380463123321533 error
Epsilon 0.03: 2.4682137966156006 error
Epsilon 0.06: 10.37920093536377 error
Epsilon 0.1: 38.12845230102539 error
fgsm small: 
Epsilon 0.003: 0.014814087189733982 error
Epsilon 0.03: 0.17861822247505188 error
Epsilon 0.06: 0.4336315989494324 error
Epsilon 0.1: 0.929533839225769 error
pgd: 
Epsilon 0.003: 0.008690240792930126 error
Epsilon 0.03: 0.13644909858703613 error
Epsilon 0.06: 0.24094991385936737 error
Epsilon 0.1: 0.7786321043968201 error
p

## Gradient Information

#### Training data

In [31]:
for cur_model, name in zip([model, step_ll_model, step_ll_model_small, fgsm_model, fgsm_model_small, pgd_model_6, pgd_model, jac_norm_model, cure], ['normal', 'step ll', 'step ll small', 'fgsm', 'fgsm small', 'pgd', 'pgd small', 'grad norm', 'cure']):
    print(name + ': ' + str(gradient_information(cur_model, train_dataset, iters=50, device=device, subset_size=5000).item()))

normal: 0.08381164073944092
step ll: 0.01806812360882759
step ll small: 0.0999184250831604
fgsm: 0.11507304757833481
fgsm small: 0.03936716541647911
pgd: 0.04062598943710327
pgd small: 0.05712084844708443
grad norm: 0.1348295509815216
cure: 0.07283072918653488


#### Testing data

In [32]:
for cur_model, name in zip([model, step_ll_model, step_ll_model_small, fgsm_model, fgsm_model_small, pgd_model_6, pgd_model, jac_norm_model, cure], ['normal', 'step ll', 'step ll small', 'fgsm', 'fgsm small', 'pgd', 'pgd small', 'grad norm', 'cure']):
    print(name + ': ' + str(gradient_information(cur_model, test_dataset, iters=50, device=device, subset_size=5000).item()))

normal: 0.10207227617502213
step ll: 0.03300695866346359
step ll small: 0.0948532372713089
fgsm: 0.11606040596961975
fgsm small: 0.042634010314941406
pgd: 0.026364196091890335
pgd small: 0.03546912968158722
grad norm: 0.13850928843021393
cure: 0.0751025527715683
